# AIPI 590 - XAI | Assignment #9
### Human AI Interaction
### Shaunak Badani


[![Open In Collab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github.com/shaunak-badani/XAI/blob/main/Assignment09/Counterfactuals.ipynb)

> This notebook outlines the details of counterfactuals and how they are useful in helping Interpretability of ML models

In [ ]:
# !pip install -q alibi

In [ ]:
# !pip uninstall -y numpy && pip install -q numpy==1.26.4

In [ ]:
!pip install -q numpy==1.25.2 pandas==2.0.3 matplotlib==3.7.1 scikit-learn==1.2.2 tensorflow==2.14.1 alibi[tensorflow]==0.9.6

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
tf.get_logger().setLevel(40) # suppress deprecation messages
tf.compat.v1.disable_v2_behavior()
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.utils import to_categorical

import os
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from alibi.explainers import Counterfactual

print('TF version: ', tf.__version__)
print('Eager execution enabled: ', tf.executing_eagerly()) # False



TF version:  2.14.1
Eager execution enabled:  False


In [ ]:
california = fetch_california_housing(as_frame=True)
X = california.data.to_numpy()
target = california.target.to_numpy()
feature_names = california.feature_names

In [ ]:
california.data

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25
...,...,...,...,...,...,...,...,...
20635,1.5603,25.0,5.045455,1.133333,845.0,2.560606,39.48,-121.09
20636,2.5568,18.0,6.114035,1.315789,356.0,3.122807,39.49,-121.21
20637,1.7000,17.0,5.205543,1.120092,1007.0,2.325635,39.43,-121.22
20638,1.8672,18.0,5.329513,1.171920,741.0,2.123209,39.43,-121.32


In [ ]:
print(california.DESCR)

.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

    :Number of Instances: 20640

    :Number of Attributes: 8 numeric, predictive attributes and the target

    :Attribute Information:
        - MedInc        median income in block group
        - HouseAge      median house age in block group
        - AveRooms      average number of rooms per household
        - AveBedrms     average number of bedrooms per household
        - Population    block group population
        - AveOccup      average number of household members
        - Latitude      block group latitude
        - Longitude     block group longitude

    :Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
https://www.dcc.fc.up.pt/~ltorgo/Regression/cal_housing.html

The target variable is the median house value for California districts,
expressed in hundreds of thousands of dollars ($100,000).

This dataset was derived

## Train the model

- In the next few code blocks, we will define a neural network with the following configuration:

  - Input size: 8
  - Hidden layer 1 size: 50
  - Hidden layer 2 size: 25
  - Output layer size: 1

- We will frame it as a classification problem. The model will predict whether the house price is above overall median price or not.

In [ ]:
np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
def nn_model():
    x_in = Input(shape=(8,))
    x = Dense(50, activation='relu')(x_in)
    x = Dense(25, activation='relu')(x)
    x_out = Dense(2, activation='softmax')(x)
    nn = Model(inputs=x_in, outputs=x_out)
    nn.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])
    return nn

In [ ]:
y = np.zeros((target.shape[0]))
median_household_price = np.median(target)
y[target > median_household_price] = 1

In [ ]:
# Standardizing data
mean = X.mean(axis = 0)
std = X.std(axis = 0)

X = (X - mean) / std

In [ ]:
# Train and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)



In [ ]:
nn = nn_model()


In [ ]:

nn.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 8)]               0         
                                                                 
 dense (Dense)               (None, 50)                450       
                                                                 
 dense_1 (Dense)             (None, 25)                1275      
                                                                 
 dense_2 (Dense)             (None, 2)                 52        
                                                                 
Total params: 1777 (6.94 KB)
Trainable params: 1777 (6.94 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
nn.fit(X_train, y_train, batch_size=64, epochs=500, verbose=0)
nn.save('nn_california.h5', save_format='h5')

/usr/local/lib/python3.11/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


# Generating counterfactuals

- Now we will generate counterfactuals for a particular data point
- A counterfactual explanation is the closest feature vector to a data point that can change the predefined prediction to a predefined output.

In [ ]:
data_point = X_test[1].reshape((1,) + X_test[1].shape)
shape = data_point.shape

In [ ]:
# define model
from alibi.explainers import Counterfactual

# initialize and fit the explainer
cf = Counterfactual(nn, shape, target_proba = 1.0, tol = 1e-4, target_class = 1, max_iter = 500, feature_range = (X_train.min(axis = 0), X_train.max(axis = 0)))

explanation = cf.explain(data_point)

In [ ]:
pred_class = explanation.cf['class']

# Get the probability of the predicted class from the counterfactual explanation
proba = explanation.cf['proba'][0][pred_class]

# Print the predicted class and its probability
print(f'Counterfactual prediction: {pred_class} with probability {proba}')

# Display the counterfactual example
print(explanation.cf['X']);


Counterfactual prediction: 1 with probability 0.999934196472168
[[-0.7259716   0.10325101 -0.14551258  0.19639948  0.12402885 -0.53279895
  -0.2223604   0.05256014]]


In [ ]:
counterfactual = explanation.cf['X']

denormalized_counterfactual = (counterfactual * std + mean)

In [ ]:
denormalized_counterfactual

array([[ 2.49148784e+00,  2.99389265e+01,  5.06898514e+00,
         1.18974874e+00,  1.56593132e+03, -2.46288705e+00,
         3.51569217e+01, -1.19464401e+02]])

In [ ]:
orig = data_point * mean + std
delta = denormalized_counterfactual - orig
for i, f in enumerate(feature_names):
    if np.abs(delta[0][i]) > 1e-4:
        print(f'{f}: {delta[0][i]}')

MedInc: 3.3205945708733324
HouseAge: 14.257636628965937
AveRooms: 3.449815096087166
AveBedrms: 0.4917978017578517
Population: 257.86870627253325
AveOccup: -12.73312351945807
Latitude: 41.22643005959695
Longitude: -114.92062219605486


In [ ]:
orig

array([[-8.29106730e-01,  1.56812899e+01,  1.61917005e+00,
         6.97950939e-01,  1.30806261e+03,  1.02702365e+01,
        -6.06950835e+00, -4.54377891e+00]])

In [ ]:
pd.DataFrame(orig, columns=feature_names)


,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
0,-0.829107,15.68129,1.61917,0.697951,1308.062615,10.270236,-6.069508,-4.543779


In [ ]:
pd.DataFrame(counterfactual, columns = feature_names)

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
0,-0.725972,0.103251,-0.145513,0.196399,0.124029,-0.532799,-0.22236,0.05256


In [ ]:
from alibi.explainers import CounterfactualProto
cf = CounterfactualProto(nn, shape, use_kdtree=True, theta=10., max_iterations=1000,
                         feature_range=(X_train.min(axis=0), X_train.max(axis=0)),
                         c_init=1., c_steps=10)

cf.fit(X_train)

CounterfactualProto(meta={
  'name': 'CounterfactualProto',
  'type': ['blackbox', 'tensorflow', 'keras'],
  'explanations': ['local'],
  'params': {
              'shape': (1, 8),
              'kappa': 0.0,
              'beta': 0.1,
              'feature_range': (array([-1.77429947, -2.19618048, -1.83504572, -1.61076772, -1.25612255,
       -0.22899997, -1.44288613, -2.38599234]), array([  5.85828581,   1.85618152,  55.16323628,  51.78248741,
        30.25033022, 119.41910319,   2.95806762,   2.62528006])),
              'gamma': 0.0,
              'theta': 10.0,
              'cat_vars': None,
              'ohe': False,
              'use_kdtree': True,
              'learning_rate_init': 0.01,
              'max_iterations': 1000,
              'c_init': 1.0,
              'c_steps': 10,
              'eps': (0.001, 0.001),
              'clip': (-1000.0, 1000.0),
              'update_num_grad': 1,
              'write_dir': None,
              'is_model': True,
              '

In [ ]:
explanation = cf.explain(data_point)

In [ ]:
orig = data_point * mean + std
counterfactual = explanation.cf['X']

denormalized_counterfactual = (counterfactual * std + mean)
delta = denormalized_counterfactual - orig
for i, f in enumerate(feature_names):
    if np.abs(delta[0][i]) > 1e-4:
        print(f'{f}: {delta[0][i]}')

MedInc: 3.3604066849409167
HouseAge: 14.318710175508507
AveRooms: 3.4202134950662213
AveBedrms: 0.4955422139514951
Population: 256.93738911248283
AveOccup: -8.498322959257356
Latitude: 41.15533127646289
Longitude: -114.9162210860683


In [ ]:
pd.DataFrame(denormalized_counterfactual, columns = feature_names)

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
0,2.5313,30.0,5.039384,1.193493,1565.000004,1.771914,35.085823,-119.46


# AI Usage:

- No AI was used during the writing of this notebook. References were taken from online tutorials and from the XAI github repository.